## DIA 096: Generar Reporte PDF del Historial Filtrado por Fechas (Flask + ReportLab)

Hoy aprenderás a generar un archivo PDF con los datos filtrados por fecha del usuario logueado.
Esto es ideal para generar reportes más formales que se pueden enviar por correo o guardar.

✅ ¿Qué aprenderás hoy?

📌 Filtrar datos por rango de fechas.

📌 Crear documentos PDF con ReportLab.

📌 Descargar el archivo dinámicamente.

🧩 1. Instalar ReportLab (si no lo tienes)
bash
Copiar
Editar
pip install reportlab
🧩 2. Ruta Nueva en app.py → /exportar-pdf-filtrado
python
Copiar
Editar
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import io

@app.route('/exportar-pdf-filtrado', methods=['GET', 'POST'])
@login_required
def exportar_pdf_filtrado():
    if request.method == 'POST':
        fecha_inicio = request.form['fecha_inicio']
        fecha_fin = request.form['fecha_fin']

        if not fecha_inicio or not fecha_fin:
            return "⚠️ Ambas fechas son obligatorias", 400

        try:
            f1 = datetime.strptime(fecha_inicio, '%Y-%m-%d')
            f2 = datetime.strptime(fecha_fin, '%Y-%m-%d')

            registros = Registro.query.filter(
                Registro.user_id == current_user.id,
                Registro.fecha >= f1,
                Registro.fecha <= f2
            ).order_by(Registro.fecha.desc()).all()

            if not registros:
                return "⚠️ No se encontraron registros en ese rango", 404

            buffer = io.BytesIO()
            p = canvas.Canvas(buffer, pagesize=letter)
            width, height = letter

            p.setFont("Helvetica-Bold", 14)
            p.drawString(50, height - 50, f"Reporte de Predicciones de {current_user.username}")
            p.setFont("Helvetica", 12)
            p.drawString(50, height - 70, f"Desde {fecha_inicio} hasta {fecha_fin}")

            y = height - 100
            p.setFont("Helvetica", 10)
            p.drawString(50, y, "Fecha        | F1 | F2 | F3 | F4 | Predicción")
            y -= 15
            p.line(50, y, width - 50, y)
            y -= 15

            for r in registros:
                linea = f"{r.fecha.strftime('%Y-%m-%d %H:%M')}  | {r.f1}  | {r.f2}  | {r.f3}  | {r.f4}  | {r.prediccion}"
                p.drawString(50, y, linea)
                y -= 15
                if y < 50:
                    p.showPage()
                    y = height - 50

            p.save()
            buffer.seek(0)

            return send_file(
                buffer,
                mimetype='application/pdf',
                as_attachment=True,
                download_name=f"reporte_{fecha_inicio}_a_{fecha_fin}.pdf"
            )

        except Exception as e:
            return f"❌ Error al generar el PDF: {str(e)}", 500

    return render_template('exportar_pdf_filtrado.html')
🖼️ 3. Template: exportar_pdf_filtrado.html
html
Copiar
Editar
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Exportar PDF Filtrado</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="bg-light">
<div class="container mt-5">
    <h2 class="text-center mb-4">🧾 Exportar Historial a PDF</h2>
    <form method="POST" class="row g-3">
        <div class="col-md-5">
            <label class="form-label">Desde:</label>
            <input type="date" class="form-control" name="fecha_inicio" required>
        </div>
        <div class="col-md-5">
            <label class="form-label">Hasta:</label>
            <input type="date" class="form-control" name="fecha_fin" required>
        </div>
        <div class="col-md-2 align-self-end">
            <button type="submit" class="btn btn-danger w-100">Exportar PDF</button>
        </div>
    </form>
    <a href="/mi-historial" class="btn btn-outline-secondary mt-4">← Volver</a>
</div>
</body>
</html>
🔗 4. Botón de acceso desde otras vistas
html
Copiar
Editar
<a href="/exportar-pdf-filtrado" class="btn btn-outline-danger">📄 Exportar PDF</a>
🔍 Líneas Clave Explicadas
canvas.Canvas(...): crea un archivo PDF en memoria.

drawString(...): escribe texto en el PDF.

buffer = io.BytesIO(): almacenamiento temporal del archivo.

send_file(...): descarga el archivo generado.

